### Data preparation

In [2]:
import pandas as pd 
import numpy as np

data = pd.read_csv("TrainDataset_AfterMulti.csv")
X = data.iloc[:, :-1]
y = np.array(data.iloc[: , -1])

### 2、Train GWO-RF

In [178]:
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

In [2]:
X_train = pd.read_csv("RF_X_train.csv")
X_test = pd.read_csv("RF_X_test.csv")
X_train = X_train.iloc[:,1:]
X_test = X_test.iloc[:,1:]
temp1 = pd.read_csv("RF_Split_y_t_v.csv")
y_train = temp1.iloc[:,1]
y_test = temp1.iloc[:162,2]

In [4]:
# load the trained model
# from sklearn.externals import joblib
# clf1 = joblib.load('model/GWO_RF.pkl')

In [3]:
from sklearn.metrics import mean_squared_error
Train_y_pred = clf1.predict(X_train)
Vlid_y_pred = clf1.predict(X_test)
mse_train = np.sqrt(mean_squared_error(y_train, Train_y_pred))
mse_valid = np.sqrt(mean_squared_error(y_test, Vlid_y_pred))
print("Molde with pure tree, Train MSE: {} Test MSE: {}".format(mse_train, mse_valid))

In [6]:
from sklearn.ensemble import RandomForestClassifier
clf1 = RandomForestClassifier(n_estimators=14, min_samples_leaf=14 ,random_state=None).fit(X_train, y_train)
# cv_scores = cross_val_score(clf1, X_test, y_test,cv =5,scoring = 'accuracy')
from sklearn.metrics import mean_squared_error
Train_y_pred = clf1.predict(X_train)
Vlid_y_pred = clf1.predict(X_test)
# cv_scores.mean()
mse_train = np.sqrt(mean_squared_error(y_train, Train_y_pred))
mse_valid = np.sqrt(mean_squared_error(y_test, Vlid_y_pred))
print("Molde with pure tree, Train MSE: {} Test MSE: {}".format(mse_train, mse_valid))

In [7]:
Vlid_y_prob = clf1.predict_proba(X_test)[:,1]
Train_y_prob = clf1.predict_proba(X_train)[:,1]

### 3、Plot learning curve

In [6]:
from sklearn.model_selection import ShuffleSplit
cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=0)

In [7]:
from sklearn.model_selection import learning_curve
train_sizes1, train_scores1, valid_scores1 = learning_curve( clf1, X_train, y_train, train_sizes=np.linspace(0.0000001, 1.0, 500), cv=cv, scoring='accuracy')
# train_error = 1 - np.mean(train_scores1,axis=1)
# test_error = 1 - np.mean(valid_scores1, axis=1)
train_mean = np.mean(train_scores1,axis=1)
train_std = np.std(train_scores1,axis=1)
valid_mean = np.mean(valid_scores1, axis=1)
valid_std = np.std(valid_scores1, axis=1)

In [ ]:
from matplotlib.pyplot import MultipleLocator
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(5,3))
plt.style.use('seaborn-notebook')
plt.plot(train_sizes1, train_mean, color='blue',  markersize=5, label='Training score Std=±{0:.4f}'.format(train_std[6:].mean()))
plt.fill_between(train_sizes1,train_mean+train_std,train_mean-train_std,alpha=0.15,color='blue')
plt.plot(train_sizes1, valid_mean, color='red',  markersize=5, label='Cross-validation score Std=±{0:.4f}'.format(valid_std[6:].mean()))
plt.fill_between(train_sizes1,valid_mean+train_std,valid_mean-train_std,alpha=0.15,color='red')
# plt.grid()
plt.rcParams['xtick.direction']='in'
plt.rcParams['ytick.direction']='in'
plt.xlabel('Number of training samples')
plt.ylabel('Accuracy')
plt.ylim([0.35,1.02])
# plt.legend(loc='lower right')
# plt.show()
plt.savefig('RF-leaningCurv600dpi.png', dpi=600)

### 4、Metrics

In [8]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, cohen_kappa_score, mean_squared_error, mean_absolute_error
tn, fp, fn, tp= confusion_matrix(y_train, Train_y_pred, labels=[0,1]).ravel()
# tn, fp, fn, tp = confusion_matrix(y_test, Vlid_y_pred, labels=[0,1]).ravel()
print(tn, fp, fn, tp)
# print(accuracy_score(y_test, Vlid_y_pred))
print(accuracy_score(y_train, Train_y_pred))
print(classification_report(y_train, Train_y_pred))
# print(classification_report(y_test, Vlid_y_pred))
tpr = tp/(tp+fn)
fpr = fp/(fp+tn)
print('TPR:%s'%tpr,' FPR:%s'%fpr)
# cohen_kappa_score(y_test, Vlid_y_pred)
cohen_kappa_score(y_train, Train_y_pred)
print("kappa:%s"%cohen_kappa_score(y_train, Train_y_pred))
rmse_train = np.sqrt(mean_squared_error(y_train, Train_y_pred))
rmse_valid = np.sqrt(mean_squared_error(y_test, Vlid_y_pred))
print("Molde with pure tree, Train RMSE: {} Valid RMSE: {}".format(rmse_train, rmse_valid))
mse_train = mean_squared_error(y_train, Train_y_pred)
mse_valid = mean_squared_error(y_test, Vlid_y_pred)
print("Molde with pure tree, Train MSE: {} Valid MSE: {}".format(mse_train, mse_valid))
# mae_train = median_absolute_error(y_train, Train_y_pred)
# mae_valid = median_absolute_error(y_test, Vlid_y_pred)
# print("Molde with pure tree, Train MAE: {} Valid MAE: {}".format(mae_train, mae_valid))

In [10]:
from sklearn.metrics import roc_curve, auc
import matplotlib as mpl
import matplotlib.pyplot as plt
# FPR, TPR, thresholds = roc_curve(y_train, Train_y_prob)
FPR, TPR, thresholds = roc_curve(y_test, Vlid_y_prob)
roc_auc = auc(FPR, TPR)
plt.title('ROC')
plt.plot(FPR, TPR, 'b', label='AUC = %0.4f'%roc_auc)
plt.legend(loc='best')
plt.plot([0,1], [0,1],'r--')
plt.ylabel('TPR')
plt.xlabel('FPR')

## 5、Save ROC

In [11]:
fpr = pd.DataFrame(FPR)
tpr = pd.DataFrame(TPR)
pd.concat([fpr, tpr], axis=1)

# 6、Save GWO-RF model and predict

In [49]:
# 方法一--joblib
from sklearn.externals import joblib
# import time
# timenow = time.strftime('%Y_%m_%d_%H:%M:%S', time.localtime(time.time()))
file = 'model/GWO_RF.pkl'
joblib.dump(clf1, file)

['model/GWO_RF.pkl']

In [12]:
joblib.load('model/GWO_RF.pkl')

In [187]:
data_random = pd.read_csv("RandomPoints.csv")
X_random = data_random.iloc[:, :]
data_random.head()
data_random.shape

(65000, 13)

In [188]:
x_pred = clf1.predict(X_random)
x_pred_prob = clf1.predict_proba(X_random)
a = pd.DataFrame(x_pred)
b = pd.DataFrame(x_pred_prob)
c = pd.concat([a, b],axis=1)
c.to_csv('65000pred-RF.csv')